In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from langchain_mcp_adapters.client import MultiServerMCPClient

github_pat = os.getenv("GITHUB_PAT")

mcp_client = MultiServerMCPClient({   
    "github": {
      "url": "https://api.githubcopilot.com/mcp/",  # GitHub Copilot MCP API 엔드포인트
      "headers": {
        "Authorization": f"Bearer {github_pat}"  # Bearer 토큰 인증
      },
      "transport": "streamable_http"  # 새로운 표준 transport 방식
    }
})

In [3]:
tool_list = await mcp_client.get_tools()

In [4]:
tool_list

[StructuredTool(name='add_comment_to_pending_review', description="Add review comment to the requester's latest pending pull request review. A pending review needs to already exist to call this (check with the user if not sure).", args_schema={'properties': {'body': {'description': 'The text of the review comment', 'type': 'string'}, 'line': {'description': 'The line of the blob in the pull request diff that the comment applies to. For multi-line comments, the last line of the range', 'type': 'number'}, 'owner': {'description': 'Repository owner', 'type': 'string'}, 'path': {'description': 'The relative path to the file that necessitates a comment', 'type': 'string'}, 'pullNumber': {'description': 'Pull request number', 'type': 'number'}, 'repo': {'description': 'Repository name', 'type': 'string'}, 'side': {'description': 'The side of the diff to comment on. LEFT indicates the previous state, RIGHT indicates the new state', 'enum': ['LEFT', 'RIGHT'], 'type': 'string'}, 'startLine': {'

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=tool_list,
    state_modifier=("Use the tools provided to you to answer the user's question")
)

In [7]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:

            key = list(chunk.keys())[0]
            
            if key == 'agent':
                # Agent 메시지의 내용을 가져옴. 메세지가 비어있는 경우 어떤 도구를 어떻게 호출할지 정보를 가져옴
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print(f"'agent': '{content}'")
            
            elif key == 'tools':
                # 도구 메시지의 내용을 가져옴
                for tool_msg in chunk['tools']['messages']:
                    print(f"'tools': '{tool_msg.content}'")
            
            results.append(chunk)
        return results
    except Exception as e:
        print(f"Error processing stream: {e}")
        return results

In [ ]:
from langchain_core.messages import HumanMessage

query = """깃헙의 Commit 목록을 확인하고 commit 관련 정보를 작성해주세요. 

URL:

"""

stream_generator = agent.astream({'messages': [HumanMessage(content=query)]})


all_chunks = await process_stream(stream_generator)


if all_chunks:
    final_result = all_chunks[-1]
    print(final_result)

'agent': '{'tool_calls': [{'id': 'call_B0XTJVSyYVvErIbry77g84OO', 'function': {'arguments': '{"owner":"rmfhrm","repo":"AI_agent_work","sha":"618fd1b10d649d5385f4ebe58b7e916968ea15ba"}', 'name': 'get_commit'}, 'type': 'function'}], 'refusal': None}'
'tools': '{"sha":"618fd1b10d649d5385f4ebe58b7e916968ea15ba","html_url":"https://github.com/rmfhrm/AI_agent_work/commit/618fd1b10d649d5385f4ebe58b7e916968ea15ba","commit":{"message":"에이전트_음성,이미지\u003e생성형 ai 활용","author":{"name":"rmfhrm","email":"rmfhrm9@gmail.com","date":"2025-08-28T08:35:58Z"},"committer":{"name":"rmfhrm","email":"rmfhrm9@gmail.com","date":"2025-08-28T08:35:58Z"}},"author":{"login":"rmfhrm","id":221989638,"profile_url":"https://github.com/rmfhrm","avatar_url":"https://avatars.githubusercontent.com/u/221989638?v=4"},"committer":{"login":"rmfhrm","id":221989638,"profile_url":"https://github.com/rmfhrm","avatar_url":"https://avatars.githubusercontent.com/u/221989638?v=4"},"stats":{"additions":8471,"total":8471},"files":[{"filen